In [27]:
#libraries from nlp

import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer=PorterStemmer()
import tensorflow as tf 
import json
import numpy as np
import random

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
import pandas as pd
intents=pd.read_json('intents.json')

In [29]:
intents

,intents
0,"{'tag': 'greeting', 'patterns': ['Hi', 'How ar..."
1,"{'tag': 'goodbye', 'patterns': ['Bye', 'See yo..."
2,"{'tag': 'thanks', 'patterns': ['Thanks', 'Than..."
3,"{'tag': 'chatbot', 'patterns': ['Who built thi..."
4,"{'tag': 'location', 'patterns': ['What is your..."
5,"{'tag': 'connect', 'patterns': ['Give me your ..."
6,"{'tag': 'movies', 'patterns': ['Which is your ..."
7,"{'tag': 'about', 'patterns': ['Who are you?', ..."


In [30]:
words =[]
classes=[]
documents=[]
ignore=['?']
#loop through each sentence in the intent's paterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokanize each and every word in the sentence
        w=nltk.word_tokenize(pattern)
        #add word to the word list
        words.extend(w)
        #add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [31]:
#perform stemming and lower each word as well as remove duplicates
words= [stemmer.stem(w.lower()) for w in words if w not in ignore]
words= sorted(list(set(classes)))
#remove duplicate classes
classes= sorted(list(set(words)))

print (len(documents), 'documents')
print (len(classes), 'classes', classes)
print (len(words),'unique stemmed words', words)

0 documents
8 classes ['about', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'movies', 'thanks']
8 unique stemmed words ['about', 'chatbot', 'connect', 'goodbye', 'greeting', 'location', 'movies', 'thanks']


In [32]:
#create training data
training=[]
output=[]
# create an empty array for output
output_empty= [0]* len(classes)

# create taring set, bag of words for each sentence
for doc in documents:
    #initialize bag of words
    bag=[]
    #list of toknized words for pattern
    pattern_words =doc[0]
    #stemming each word
    pattern_words=[stemmer.stem(word.lower()) for word in pattern_words]
    #create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    #output is '1' for current tag and '0' for rest of other tags
    output_row= list(output_empty)
    output_row[classes.index(doc[1])]=1
    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training= np.array(training)

# creating training lists
train_x= list(training[:0])
train_y=list(training[:1])

In [51]:
model =tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]))))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[1]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', matrics=['accuracy'])

IndexError: list index out of range

In [54]:
model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=8, verbose=1)
model.save('model.pk1')

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.